In [ ]:
# Remember to install tianshou first
!pip install tianshou==0.4.8
!pip install gym

# Overview
From its literal meaning, we can easily know that the Collector in Tianshou is used to collect training data. More specificly, the Collector controls the interaction between Policy (agent) and the environment. It also helps save the interaction data into the ReplayBuffer and returns episode statistics.

<center>
</img>
</center>



# Usages
Collector can be used both for training (data collecting) and evaluation in Tianshou.

## Policy evaluation
We need to evaluate our trained policy from time to time in DRL experiments. Collector can help us with this.

First we have to initialise a Collector with an (vectorized) environment and a given policy (agent).

In [ ]:
import gym
import numpy as np
import torch

from tianshou.data import Collector
from tianshou.env import DummyVectorEnv
from tianshou.policy import PGPolicy
from tianshou.utils.net.common import Net
from tianshou.utils.net.discrete import Actor

import warnings
warnings.filterwarnings('ignore')

env = gym.make("CartPole-v0")
test_envs = DummyVectorEnv([lambda: gym.make("CartPole-v0") for _ in range(2)])

# model
net = Net(env.observation_space.shape, hidden_sizes=[16,])
actor = Actor(net, env.action_space.shape)
optim = torch.optim.Adam(actor.parameters(), lr=0.0003)

policy = PGPolicy(actor, optim, dist_fn=torch.distributions.Categorical)
test_collector = Collector(policy, test_envs)

Now we would like to collect 9 episodes of data to test how our initialised Policy performs.

In [ ]:
collect_result = test_collector.collect(n_episode=9)
print(collect_result)
print("Rewards of 9 episodes are {}".format(collect_result["rews"]))
print("Average episode reward is {}.".format(collect_result["rew"]))
print("Average episode length is {}.".format(collect_result["len"]))

Now we wonder what is the performance of a random policy.

In [ ]:
# Reset the collector
test_collector.reset()
collect_result = test_collector.collect(n_episode=9, random=True)
print(collect_result)
print("Rewards of 9 episodes are {}".format(collect_result["rews"]))
print("Average episode reward is {}.".format(collect_result["rew"]))
print("Average episode length is {}.".format(collect_result["len"]))

Seems that an initialised policy performs even worse than a random policy without any training.

## Data Collecting
Data collecting is mostly used during training, when we need to store the collected data in a ReplayBuffer.

In [ ]:
from tianshou.data import VectorReplayBuffer
train_env_num = 4
buffer_size = 100
train_envs = DummyVectorEnv([lambda: gym.make("CartPole-v0") for _ in range(train_env_num)])
replaybuffer = VectorReplayBuffer(buffer_size, train_env_num)

train_collector = Collector(policy, train_envs, replaybuffer)

Now we can collect 50 steps of data, which will be automatically saved in the replay buffer. You can still choose to collect a certain number of episodes rather than steps. Try it yourself.

In [ ]:
print(len(replaybuffer))
collect_result = train_collector.collect(n_step=50)
print(len(replaybuffer))
print(collect_result)

In [ ]:
for i in range(13):
    print(i, replaybuffer.next(i))

In [ ]:
replaybuffer.sample(10)

# Further Reading
The above collector actually collects 52 data at a time because 52 % 4 = 0. There is one asynchronous collector which allows you collect exactly 50 steps. Check the [documentation](https://tianshou.readthedocs.io/en/master/api/tianshou.data.html#asynccollector) for details.